In [1]:
!pip install --upgrade --quiet transformers bert-score pandas tqdm datasets accelerate peft bitsandbytes pillow trl --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 21.6 MB/s eta 0:00:00


In [2]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 22.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.3.10.19:
      Successfully uninstalled nvidia-curand-cu12-10.3.10.19
  Attempt

In [3]:
import pandas as pd
import os
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering
from bert_score import score

2025-05-14 18:32:01.342678: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747247521.577807      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747247521.643002      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
df1 = pd.read_csv("/kaggle/input/vqa-test-dataset/blip_vqa_test.csv")
df2 = pd.read_csv("/kaggle/input/vqa-test-dataset/blip_vqa_val.csv")

df = pd.concat([df1, df2], ignore_index=True)

In [5]:
device    = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model     = BlipForQuestionAnswering.from_pretrained("/kaggle/input/vrmp2-blip-finetuned-model/vqa-blip-base-final-model").to(device)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [6]:
results = []

for _, row in df.iterrows():
    image_path = os.path.join("/kaggle/input/abo-small/images/small", row["image_path"])
    image = Image.open(image_path).convert("RGB")
    inputs = processor(image, row["question"], return_tensors="pt").to(device)
    out = model.generate(**inputs)
    prediction = processor.decode(out[0], skip_special_tokens=True).strip()

    results.append({
        "ground_truth":  str(row["answer"]).strip(),
        "prediction":    prediction
    })

results_df = pd.DataFrame(results)

results_df.to_csv("BLIP_FineTuned_Prediction.csv", index=False)

In [7]:
results_df

,ground_truth,prediction
0,Nickel,silver
1,Shower,showerhead
2,Round,round
3,Five,three
4,Hose,chain
...,...,...
18751,Bag,bag
18752,Strap,strap
18753,Strap,strap
18754,AmazonBasics,amazon


In [8]:
# before computing any metrics

y_true = results_df['ground_truth'].fillna('').astype(str)
y_pred = results_df['prediction'].fillna('').astype(str)

In [9]:
labels = sorted(set(y_true).union(set(y_pred)))

In [10]:
# 1. Exact Match Score

import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    cohen_kappa_score,
    matthews_corrcoef,
    balanced_accuracy_score
)

acc = accuracy_score(y_true, y_pred)

print(f"Exact-match Accuracy: {acc:.4f}")

Exact-match Accuracy: 0.0035


In [11]:
# 2. BERTScore

from bert_score import score as bert_score

P, R, F1 = bert_score(
    cands = y_pred.tolist(),
    refs = y_true.tolist(),
    lang = "en",
    rescale_with_baseline = True
)

print(f"\nBERTScore →  P: {P.mean():.4f},  R: {R.mean():.4f},  F1: {F1.mean():.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



BERTScore →  P: 0.7913,  R: 0.6718,  F1: 0.7267


In [12]:
# 3. Semantic cosine similarity (via SentenceTransformers)

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

st_model = SentenceTransformer("all-MiniLM-L6-v2")
emb_true = st_model.encode(y_true.tolist(), convert_to_numpy = True)
emb_pred = st_model.encode(y_pred.tolist(), convert_to_numpy = True)

# cosine_similarity gives an NxN matrix; we take its diagonal
cos_mat     = cosine_similarity(emb_true, emb_pred)
cos_scores  = np.diag(cos_mat)
print(f"\nAvg. semantic cosine similarity: {cos_scores.mean():.4f}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/587 [00:00<?, ?it/s]

Batches:   0%|          | 0/587 [00:00<?, ?it/s]


Avg. semantic cosine similarity: 0.8217
